In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
# !pip install nyoka
from nyoka import xgboost_to_pmml
import logging
logger = logging.getLogger(__name__)

Read and prep csv titanic data

In [ ]:
titanic = pd.read_csv("train.csv")
titanic.drop(['Name', 'Ticket','Cabin','PassengerId'], axis=1, inplace=True) 
features = titanic.columns
titanic=pd.get_dummies(titanic)

#rearrage columns
survived = titanic.pop('Survived')
titanic.insert(10, 'Survived', survived)

display(titanic)

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived
0,3,22.0,1,0,7.2500,0,1,0,0,1,0
1,1,38.0,1,0,71.2833,1,0,1,0,0,1
2,3,26.0,0,0,7.9250,1,0,0,0,1,1
3,1,35.0,1,0,53.1000,1,0,0,0,1,1
4,3,35.0,0,0,8.0500,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,1,0,0,1,0
887,1,19.0,0,0,30.0000,1,0,0,0,1,1
888,3,NaN,1,2,23.4500,1,0,0,0,1,0
889,1,26.0,0,0,30.0000,0,1,1,0,0,1


Split train data

In [ ]:
target = 'Survived'
y = titanic.pop(target)
features = titanic.columns

X_train, X_test, y_train, y_test=train_test_split(titanic, y, test_size=0.30, random_state=42)


Training pipeline object

In [ ]:
pipeline_obj = Pipeline([("imp", SimpleImputer(strategy="median")),
           ("gbc", XGBClassifier(use_label_encoder=False))])
pipeline_obj.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('imp',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('gbc',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               use_label_encoder=False, verbosity=1))],
         verbose=False)

Determine predictions from pipeline object

In [ ]:
predictions = pipeline_obj.predict(X_test)
print(predictions)

[0 0 0 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1
 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 0 0 0 0 0 1
 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1
 0 0 0 1 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 1 1 0 1 0 0 0 0 0]


Determine name and create pmml file with  ```xgboost_to_pmml()```

In [ ]:
f_name = "XGB_titanic.pmml"
xgboost_to_pmml(pipeline_obj, features, target, f_name)